# Traffic Simulation for Monitoring Dashboard

Send periodic inference requests to the deployed endpoint to simulate
real-world traffic patterns and populate CloudWatch metrics and Model Monitor data.

## Setup

In [106]:
import boto3
import sagemaker
import pandas as pd
import numpy as np
import time
from datetime import datetime
from sqlalchemy import create_engine
from sagemaker.predictor import Predictor
from sklearn.metrics import accuracy_score, roc_auc_score

sess   = sagemaker.Session()
region = boto3.Session().region_name
bucket = sess.default_bucket()

s3_prefix = "aai540/model/xgboost-binary"

FEATURE_COLS = [
    "duration", "pkt_total", "bytes_total",
    "pkt_fwd", "pkt_bwd", "bytes_fwd", "bytes_bwd",
    "pkt_rate", "byte_rate", "bytes_per_pkt",
    "pkt_ratio", "byte_ratio",
]
LABEL_COL = "label"

ENDPOINT_NAME = "ids-xgboost-binary-monitor"

print(f"Region   : {region}")
print(f"Endpoint : {ENDPOINT_NAME}")

Region   : us-east-1
Endpoint : ids-xgboost-binary-monitor


## Load Production Data

Using the production holdout dataset (40% split) which represents truly unseen data
that wasn't used for training, validation, or testing. This simulates real production
inference traffic.

In [107]:
# load production data from Athena (40% holdout and unseen by model)
database_name = "aai540_eda"
engine = create_engine(
    f"awsathena+rest://@athena.{region}.amazonaws.com:443/{database_name}",
    connect_args={
        "s3_staging_dir": f"s3://{bucket}/athena/staging/",
        "region_name": region,
    },
)

columns = ", ".join([LABEL_COL] + FEATURE_COLS)
query = f"""
SELECT {columns}
FROM {database_name}.dataset_split
WHERE data_split = 'prod'
"""
print("Loading production data from Athena...")
df_prod = pd.read_sql(query, engine)
print(f"Production rows loaded: {len(df_prod):,}")

X_test = df_prod[FEATURE_COLS].values
y_true = df_prod[LABEL_COL].values

Loading production data from Athena...
Production rows loaded: 199,627


## Connect to Endpoint

In [108]:
predictor = Predictor(
    endpoint_name=ENDPOINT_NAME,
    sagemaker_session=sess,
    serializer=sagemaker.serializers.CSVSerializer(),
    deserializer=sagemaker.deserializers.CSVDeserializer(),
)
print(f"Connected to endpoint: {ENDPOINT_NAME}")

Connected to endpoint: ids-xgboost-binary-monitor


## Traffic Simulation Configuration

In [109]:
# Configuration
DURATION_MINUTES = 60       # Total simulation duration (1 hour)
INTERVAL_SECONDS = 60        # Time between batches
BATCH_SIZE_MIN = 50          # Minimum samples per batch
BATCH_SIZE_MAX = 200         # Maximum samples per batch
BATCH_SIZE_AVG = 100         # Average samples per batch

total_iterations = (DURATION_MINUTES * 60) // INTERVAL_SECONDS

print("Traffic Simulation Configuration")

print(f"  Duration        : {DURATION_MINUTES} minutes ({DURATION_MINUTES/60:.1f} hours)")
print(f"  Interval        : {INTERVAL_SECONDS} seconds between batches")
print(f"  Batch size      : {BATCH_SIZE_MIN}-{BATCH_SIZE_MAX} samples (avg ~{BATCH_SIZE_AVG})")
print(f"  Total batches   : {total_iterations}")
print(f"  Est. requests   : ~{total_iterations * BATCH_SIZE_AVG:,} (variable load)")
print(f"\nStart time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Traffic Simulation Configuration
  Duration        : 60 minutes (1.0 hours)
  Interval        : 60 seconds between batches
  Batch size      : 50-200 samples (avg ~100)
  Total batches   : 60
  Est. requests   : ~6,000 (variable load)

Start time: 2026-02-17 17:59:19


## Run Simulation

This cell will run for the configured duration

In [110]:
all_predictions = []
all_actuals = []
iteration = 0

try:
    for i in range(total_iterations):
        iteration = i + 1
        
        # Variable batch size to simulate realistic traffic patterns
        # Create periods of high/medium/low traffic
        hour_position = (i / total_iterations) * DURATION_MINUTES / 60
        
        # Sine wave pattern for natural variation + random noise
        traffic_cycle = np.sin(hour_position * 2 * np.pi) * 0.3 + 0.7  # 0.4 to 1.0
        random_variation = np.random.uniform(0.85, 1.15)  # ±15% random noise
        
        batch_size = int(BATCH_SIZE_AVG * traffic_cycle * random_variation)
        batch_size = max(BATCH_SIZE_MIN, min(BATCH_SIZE_MAX, batch_size))  # Clamp
        
        # Random sample from test set
        indices = np.random.choice(len(X_test), size=batch_size, replace=False)
        batch_X = X_test[indices]
        batch_y = y_true[indices]
        
        # Send predictions
        start_time = time.time()
        response = predictor.predict(batch_X)
        latency = (time.time() - start_time) * 1000  # ms
        
        # Parse predictions
        y_prob = np.array([float(row[0]) for row in response])
        y_pred = (y_prob >= 0.5).astype(int)
        
        # Track results
        all_predictions.extend(y_pred)
        all_actuals.extend(batch_y)
        
        # Compute running metrics
        batch_accuracy = accuracy_score(batch_y, y_pred)
        cumulative_accuracy = accuracy_score(all_actuals, all_predictions)
        
        # Progress update
        timestamp = datetime.now().strftime('%H:%M:%S')
        elapsed = i * INTERVAL_SECONDS
        remaining = (total_iterations - i - 1) * INTERVAL_SECONDS
        
        print(
            f"[{timestamp}] Batch {iteration}/{total_iterations} | "
            f"Samples: {batch_size:3d} | Latency: {latency:.0f}ms | "
            f"Batch Acc: {batch_accuracy:.3f} | "
            f"Cumulative Acc: {cumulative_accuracy:.4f} | "
            f"Remaining: {remaining//60}m {remaining%60}s"
        )
        
        # Wait before next batch (unless last iteration)
        if i < total_iterations - 1:
            time.sleep(INTERVAL_SECONDS)
            
except KeyboardInterrupt:
    print(f"\nSimulation interrupted at batch {iteration}/{total_iterations}")

# Final summary
print("Simulation Complete")
print(f"  Batches sent    : {iteration}/{total_iterations}")
print(f"  Total samples   : {len(all_predictions):,}")
print(f"  Overall accuracy: {accuracy_score(all_actuals, all_predictions):.4f}")
print(f"  End time        : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

[17:59:19] Batch 1/60 | Samples:  67 | Latency: 71ms | Batch Acc: 0.149 | Cumulative Acc: 0.1493 | Remaining: 59m 0s
[18:00:19] Batch 2/60 | Samples:  83 | Latency: 15ms | Batch Acc: 0.205 | Cumulative Acc: 0.1800 | Remaining: 58m 0s
[18:01:19] Batch 3/60 | Samples:  85 | Latency: 17ms | Batch Acc: 0.247 | Cumulative Acc: 0.2043 | Remaining: 57m 0s
[18:02:19] Batch 4/60 | Samples:  90 | Latency: 16ms | Batch Acc: 0.322 | Cumulative Acc: 0.2369 | Remaining: 56m 0s
[18:03:19] Batch 5/60 | Samples:  71 | Latency: 15ms | Batch Acc: 0.324 | Cumulative Acc: 0.2525 | Remaining: 55m 0s
[18:04:19] Batch 6/60 | Samples:  84 | Latency: 77ms | Batch Acc: 0.298 | Cumulative Acc: 0.2604 | Remaining: 54m 0s
[18:05:19] Batch 7/60 | Samples:  83 | Latency: 18ms | Batch Acc: 0.229 | Cumulative Acc: 0.2558 | Remaining: 53m 0s
[18:06:19] Batch 8/60 | Samples:  90 | Latency: 19ms | Batch Acc: 0.211 | Cumulative Acc: 0.2496 | Remaining: 52m 0s
[18:07:19] Batch 9/60 | Samples:  83 | Latency: 15ms | Batch Acc

## Quick Dashboard Link

In [111]:
dashboard_url = (
    f"https://{region}.console.aws.amazon.com/cloudwatch/home"
    f"?region={region}#dashboards/dashboard/IDS-XGBoost-Monitoring"
)
print("CloudWatch Dashboard:")
print(dashboard_url)

CloudWatch Dashboard:
https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#dashboards/dashboard/IDS-XGBoost-Monitoring
